# Hadoop Short Course


## 1. Hadoop Distributed File System

Hadoop Distributed File System (HDFS)

HDFS is the primary distributed storage used by Hadoop applications. A HDFS cluster primarily consists of a NameNode that manages the file system metadata and DataNodes that store the actual data. The [HDFS Architecture Guide](http://hadoop.apache.org/docs/r2.7.1/hadoop-project-dist/hadoop-hdfs/HdfsDesign.html) describes HDFS in detail. To learn more about the interaction of users and administrators with HDFS, please refer to [HDFS User Guide](http://hadoop.apache.org/docs/r2.7.1/hadoop-project-dist/hadoop-hdfs/HdfsUserGuide.html). 

All HDFS commands are invoked by the **bin/hdfs** script. Running the hdfs script without any arguments prints the description for all commands. For all the commands, please refer to [HDFS Commands Reference](http://hadoop.apache.org/docs/r2.7.1/hadoop-project-dist/hadoop-hdfs/HDFSCommands.html)

### Start HDFS

In [1]:
hadoop_root = '/home/ubuntu/shortcourse/hadoop-2.7.1/'
hadoop_start_hdfs_cmd = hadoop_root + 'sbin/start-dfs.sh'
hadoop_stop_hdfs_cmd = hadoop_root + 'sbin/stop-dfs.sh'

In [2]:
# start the hadoop distributed file system
! {hadoop_start_hdfs_cmd}

15/11/03 08:28:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /home/ubuntu/shortcourse/hadoop-2.7.1/logs/hadoop-ubuntu-namenode-ubuntu.out
localhost: starting datanode, logging to /home/ubuntu/shortcourse/hadoop-2.7.1/logs/hadoop-ubuntu-datanode-ubuntu.out
localhost: Java HotSpot(TM) Server VM warning: You have loaded library /home/ubuntu/shortcourse/hadoop-2.7.1/lib/native/libhadoop.so.1.0.0 which might have disabled stack guard. The VM will try to fix the stack guard now.
localhost: It's highly recommended that you fix the library with 'execstack -c <libfile>', or link it with '-z noexecstack'.
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /home/ubuntu/shortcourse/hadoop-2.7.1/logs/hadoop-ubuntu-secondarynamenode-ubuntu.out
0.0.0.0: Java HotSpot(TM) Server VM warning: You have loaded l

In [3]:
# show the jave jvm process summary
# You should see NamenNode, SecondaryNameNode, and DataNode
! jps

5160 NameNode
5526 SecondaryNameNode
5642 Jps
5320 DataNode


**Normal file operations and data preparation for later example**

In [4]:
# list recursively everything under the root dir
! {hadoop_root + 'bin/hdfs dfs -ls -R /'}

15/11/03 08:28:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
drwxr-xr-x   - ubuntu supergroup          0 2015-10-11 14:39 /user
drwxr-xr-x   - ubuntu supergroup          0 2015-11-02 20:14 /user/ubuntu
drwxr-xr-x   - ubuntu supergroup          0 2015-11-02 20:12 /user/ubuntu/input
-rw-r--r--   1 ubuntu supergroup     167518 2015-11-02 20:12 /user/ubuntu/input/alice.txt
-rw-r--r--   1 ubuntu supergroup     717574 2015-11-02 20:12 /user/ubuntu/input/pride-and-prejudice.txt
-rw-r--r--   1 ubuntu supergroup     594933 2015-11-02 20:12 /user/ubuntu/input/sherlock-holmes.txt
drwxr-xr-x   - ubuntu supergroup          0 2015-11-02 20:13 /user/ubuntu/output
-rw-r--r--   1 ubuntu supergroup          0 2015-11-02 20:13 /user/ubuntu/output/_SUCCESS
-rw-r--r--   1 ubuntu supergroup     279999 2015-11-02 20:13 /user/ubuntu/output/part-00000
drwxr-xr-x   - ubuntu supergroup          0 2015-11-02 20:14 /user/ubuntu/

Download some files for later use.

In [5]:
# We will use three ebooks from Project Gutenberg for later example
# Pride and Prejudice by Jane Austen: http://www.gutenberg.org/ebooks/1342.txt.utf-8
! wget http://www.gutenberg.org/ebooks/1342.txt.utf-8 -O /home/ubuntu/shortcourse/data/wordcount/pride-and-prejudice.txt

# Alice's Adventures in Wonderland by Lewis Carroll: http://www.gutenberg.org/ebooks/11.txt.utf-8
! wget http://www.gutenberg.org/ebooks/11.txt.utf-8 -O /home/ubuntu/shortcourse/data/wordcount/alice.txt
    
# The Adventures of Sherlock Holmes by Arthur Conan Doyle: http://www.gutenberg.org/ebooks/1661.txt.utf-8
! wget http://www.gutenberg.org/ebooks/1661.txt.utf-8 -O /home/ubuntu/shortcourse/data/wordcount/sherlock-holmes.txt

--2015-11-03 08:28:52--  http://www.gutenberg.org/ebooks/1342.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/1342/pg1342.txt [following]
--2015-11-03 08:28:53--  http://www.gutenberg.org/cache/epub/1342/pg1342.txt
Reusing existing connection to www.gutenberg.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 717574 (701K) [text/plain]
Saving to: ‘/home/ubuntu/shortcourse/data/wordcount/pride-and-prejudice.txt’

100%[======================================>] 717,574     1003KB/s   in 0.7s   

2015-11-03 08:28:54 (1003 KB/s) - ‘/home/ubuntu/shortcourse/data/wordcount/pride-and-prejudice.txt’ saved [717574/717574]

--2015-11-03 08:28:54--  http://www.gutenberg.org/ebooks/11.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47
Connecting to www.gutenber

In [6]:
# delete existing folders
! {hadoop_root + 'bin/hdfs dfs -rm -R /user/ubuntu/*'}

15/11/03 08:29:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/11/03 08:29:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/input
15/11/03 08:29:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/output
15/11/03 08:29:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/output2


In [7]:
# create input folder
! {hadoop_root + 'bin/hdfs dfs -mkdir /user/ubuntu/input'}

15/11/03 08:29:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# copy the three books to the input folder in HDFS
! {hadoop_root + 'bin/hdfs dfs -copyFromLocal /home/ubuntu/shortcourse/data/wordcount/* /user/ubuntu/input/'}

15/11/03 08:29:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# show if the files are there
! {hadoop_root + 'bin/hdfs dfs -ls -R'}

15/11/03 08:29:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
drwxr-xr-x   - ubuntu supergroup          0 2015-11-03 08:29 input
-rw-r--r--   1 ubuntu supergroup     167518 2015-11-03 08:29 input/alice.txt
-rw-r--r--   1 ubuntu supergroup     717574 2015-11-03 08:29 input/pride-and-prejudice.txt
-rw-r--r--   1 ubuntu supergroup     594933 2015-11-03 08:29 input/sherlock-holmes.txt


## 2. WordCount Example

Let's count the single word frequency in the uploaded three books.

Start Yarn, the resource allocator for Hadoop.

In [10]:
# start the hadoop distributed file system
! {hadoop_root + 'sbin/start-yarn.sh'}

starting yarn daemons
starting resourcemanager, logging to /home/ubuntu/shortcourse/hadoop-2.7.1/logs/yarn-ubuntu-resourcemanager-ubuntu.out
localhost: starting nodemanager, logging to /home/ubuntu/shortcourse/hadoop-2.7.1/logs/yarn-ubuntu-nodemanager-ubuntu.out


In [11]:
# wordcount 1 the scripts
# Map: /home/ubuntu/shortcourse/notes/scripts/wordcount1/mapper.py
# Test locally the map script
! echo "go gators gators beat everyone go glory gators" | \
  /home/ubuntu/shortcourse/notes/scripts/wordcount1/mapper.py

go	1
gators	1
gators	1
beat	1
everyone	1
go	1
glory	1
gators	1


In [12]:
# Reduce: /home/ubuntu/shortcourse/notes/scripts/wordcount1/reducer.py
# Test locally the reduce script
! echo "go gators gators beat everyone go glory gators" | \
  /home/ubuntu/shortcourse/notes/scripts/wordcount1/mapper.py | \
  sort -k1,1 | \
  /home/ubuntu/shortcourse/notes/scripts/wordcount1/reducer.py

beat	1
everyone	1
gators	3
glory	1
go	2


In [13]:
# run them with Hadoop against the uploaded three books
cmd = hadoop_root + 'bin/hadoop jar ' + hadoop_root + 'hadoop-streaming-2.7.1.jar ' + \
    '-input input ' + \
    '-output output ' + \
    '-mapper /home/ubuntu/shortcourse/notes/scripts/wordcount1/mapper.py ' + \
    '-reducer /home/ubuntu/shortcourse/notes/scripts/wordcount1/reducer.py ' + \
    '-file /home/ubuntu/shortcourse/notes/scripts/wordcount1/mapper.py ' + \
    '-file /home/ubuntu/shortcourse/notes/scripts/wordcount1/reducer.py'

! {cmd}

15/11/03 08:29:47 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
15/11/03 08:29:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/home/ubuntu/shortcourse/notes/scripts/wordcount1/mapper.py, /home/ubuntu/shortcourse/notes/scripts/wordcount1/reducer.py] [] /tmp/streamjob5390010843768271216.jar tmpDir=null
15/11/03 08:29:48 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/11/03 08:29:48 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/11/03 08:29:48 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/11/03 08:29:49 INFO mapred.FileInputFormat: Total input paths to process : 3
15/11/03 08:29:49 INFO mapreduce.JobSubmitter: number of splits:3
15/11/03 08:29:49 INFO mapreduce.JobSubmitter: Submit

In [14]:
# list the output
! {hadoop_root + 'bin/hdfs dfs -ls -R output'}

15/11/03 08:30:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
-rw-r--r--   1 ubuntu supergroup          0 2015-11-03 08:29 output/_SUCCESS
-rw-r--r--   1 ubuntu supergroup     279999 2015-11-03 08:29 output/part-00000


In [15]:
# Let's see what's in the output file
# delete if previous results exist
! rm -rf /home/ubuntu/shortcourse/tmp/*
! {hadoop_root + 'bin/hdfs dfs -copyToLocal output/part-00000 /home/ubuntu/shortcourse/tmp/wc1-part-00000'}
! tail -n 20 /home/ubuntu/shortcourse/tmp/wc1-part-00000

15/11/03 08:30:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/11/03 08:30:44 WARN hdfs.DFSClient: DFSInputStream has been closed already
yourself--and	1
yourself.	4
yourself."	6
yourself.'	2
yourself;	1
yourself?	1
yourself?"	1
yourselves	4
yourselves,	1
yourselves?	1
youth	9
youth,	9
youth,'	3
youth?"	1
youths	1
zero,	1
zero-point,	1
zest	1
zigzag	1
zigzag,	1


## 3. Exercise: WordCount2

Count the single word frequency, where the words are given in a pattern file. 

For example, given pattern.txt file, which contains: 
    
    "a b c d"

And the input file is: 

    "d e a c f g h i a b c d". 

Then the output shoule be:

    "a 1
     b 1
     c 2
     d 2"
     
Please copy the mapper.py and reduce.py from the first wordcount example to foler "/home/ubuntu/shortcourse/notes/scripts/wordcount2/". The pattern file is given in the wordcount2 folder with name "wc2-pattern.txt"

**Hint**:
1. pass the pattern file using "-file option" and use -cmdenv to pass the file name as environment variable
2. in the mapper, read the pattern file into a set
3. only print out the words that exist in the set

In [16]:
# execise: count the words existing in the given pattern file for the three books

cmd = hadoop_root + 'bin/hadoop jar ' + hadoop_root + 'hadoop-streaming-2.7.1.jar ' + \
    '-cmdenv PATTERN_FILE=wc2-pattern.txt ' + \
    '-input input ' + \
    '-output output2 ' + \
    '-mapper /home/ubuntu/shortcourse/notes/scripts/wordcount2/mapper.py ' + \
    '-reducer /home/ubuntu/shortcourse/notes/scripts/wordcount2/reducer.py ' + \
    '-file /home/ubuntu/shortcourse/notes/scripts/wordcount2/mapper.py ' + \
    '-file /home/ubuntu/shortcourse/notes/scripts/wordcount2/reducer.py ' + \
    '-file /home/ubuntu/shortcourse/notes/scripts/wordcount2/wc2-pattern.txt'

! {cmd}

15/11/03 08:31:24 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
15/11/03 08:31:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/home/ubuntu/shortcourse/notes/scripts/wordcount2/mapper.py, /home/ubuntu/shortcourse/notes/scripts/wordcount2/reducer.py, /home/ubuntu/shortcourse/notes/scripts/wordcount2/wc2-pattern.txt] [] /tmp/streamjob6344597239144833252.jar tmpDir=null
15/11/03 08:31:25 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/11/03 08:31:25 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/11/03 08:31:25 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/11/03 08:31:26 INFO mapred.FileInputFormat: Total input paths to process : 3
15/11/03 08:31:26 INFO mapreduce.JobSubmitter: numbe

**Verify Results**

1. Copy the output file to local
2. run the following command, and compare with the downloaded output
    
    sort -nrk 2,2 part-00000  | head -n 20
    
The wc1-part-00000 is the output of the previous wordcount (wordcount1)



In [17]:
! rm -rf /home/ubuntu/shortcourse/tmp/wc2-part-00000
! {hadoop_root + 'bin/hdfs dfs -copyToLocal output2/part-00000 /home/ubuntu/shortcourse/tmp/wc2-part-00000'}
! cat /home/ubuntu/shortcourse/tmp/wc2-part-00000 | sort -nrk2,2

15/11/03 08:32:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/11/03 08:32:05 WARN hdfs.DFSClient: DFSInputStream has been closed already
the	11273
to	7594
of	6978
and	6887
a	5182
I	4533
in	3916
was	3484
that	3204
her	2428
his	2356
you	2317
it	2284
he	2148
as	2141
had	2107
with	2097
she	2095
not	2076
be	1975


In [18]:
! sort -nr -k2,2 /home/ubuntu/shortcourse/tmp/wc1-part-00000 | head -n 20

the	11273
to	7594
of	6978
and	6887
a	5182
I	4533
in	3916
was	3484
that	3204
her	2428
his	2356
you	2317
it	2284
he	2148
as	2141
had	2107
with	2097
she	2095
not	2076
be	1975
sort: write failed: standard output: Broken pipe
sort: write error


In [19]:
# stop dfs and yarn
!{hadoop_root + 'sbin/stop-yarn.sh'}
# don't stop hdfs for now, later use
# !{hadoop_stop_hdfs_cmd}

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
